# RAG (with LanceDB and LlamaParse)

## LlamaParse

In [ ]:
pdf_files = ["Vafaei402723272.pdf"]

### Load and Parse PDF file using LlamaParse

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader
import os

# set up parser
parser = LlamaParse(result_type="text", api_key=os.environ["LLAMA_PARSE_API_KEY"])

file_extractor = {".pdf": parser}

data_for_parse = SimpleDirectoryReader(input_files=pdf_files, file_extractor=file_extractor)
data_for_parse

In [ ]:
documents =data_for_parse.load_data()
documents

In [ ]:
len(documents)

### Chunk files

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=64,
    length_function=len,
    is_separator_regex=False,
)

In [ ]:
documents_list = []
page_number = 0
last_doc = None
for doc in documents:
    if last_doc is None or last_doc != doc.metadata["file_name"]:
        page_number = 1
        last_doc = doc.metadata["file_name"]
    else:
        page_number += 1

    texts = text_splitter.split_text(doc.text)
    for text in texts:
        item = {}
        item["id_"] = doc.id_
        item["text"] = text
        item["metadata_file_name"] = doc.metadata["file_name"]
        item["metadata_creation_date"] = doc.metadata["creation_date"]
        item["metadata_pagenumber"] = page_number
        documents_list.append(item)



In [ ]:
len(documents_list)

### Chunks to Pandas DataFrame

In [ ]:
import pandas as pd

df = pd.DataFrame(documents_list)
df

## LanceDB

### Connect to DB

In [ ]:
import lancedb
db = lancedb.connect(".lancedb")

### Define the embedding function

In [ ]:
from lancedb.embeddings import get_registry
embedding_model = get_registry().get("sentence-transformers").create(name="BAAI/bge-small-en-v1.5")

### Define the data model or schema

In [ ]:
#You should put HF_TOKEN in the Notebook enviroment variables
from lancedb.pydantic import LanceModel, Vector

class ChunksOfData(LanceModel):
    id_: str
    text: str = embedding_model.SourceField()
    metadata_file_name: str
    metadata_creation_date: str
    metadata_pagenumber: int
    vector: Vector(embedding_model.ndims()) = embedding_model.VectorField()

### Create table and add data

In [ ]:
def df_to_dict_batches(df: pd.DataFrame, batch_size: int = 128):
    """
    Yields data from a DataFrame in batches of dictionaries.
    Each batch is a list of dict, suitable for LanceDB ingestion.
    """
    for start_idx in range(0, len(df), batch_size):
        end_idx = start_idx + batch_size
        # Convert the batch of rows to a list of dict
        batch_dicts = df.iloc[start_idx:end_idx].to_dict(orient="records")
        yield batch_dicts

tbl = db.create_table(
    "seminar_pdf_data",
    data=df_to_dict_batches(df, batch_size=10),
    schema=ChunksOfData,
)

### Querying your table

In [ ]:
tbl = db.open_table("seminar_pdf_data")
query = "مدل های زبانی بزرگ از نظر نوع ورودی و خروجی چگونه هستند؟"
#actual = table.search(query).limit(5).to_pydantic(Words)[0]
res= tbl.search(query).limit(5).to_pandas()
res


#### Hybrid Search

In [ ]:
# query = "کاربرد سیستم های صف چیست؟"
tbl.create_fts_index('text', use_tantivy=False,replace=True)
tbl.search(query, query_type="hybrid").limit(5).to_pandas()

## RAG

In [ ]:
import os
from langchain_openai import ChatOpenAI

AVALAI_BASE_URL = "https://api.avalai.ir/v1"
GPT_MODEL_NAME = "gpt-4o-mini-2024-07-18"

gpt4o_chat = ChatOpenAI(model=GPT_MODEL_NAME,
                        base_url=AVALAI_BASE_URL,
                        api_key=os.environ["AVALAI_API_KEY"])

In [ ]:
from typing import Optional, List  # Add this line
from langchain.llms.base import LLM
from ollama import chat, ChatResponse
from pydantic import BaseModel  # Pydantic's BaseModel for field definitions

class OllamaLLM(LLM, BaseModel):
    model_name: str
    verbose: bool = False

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        if self.verbose:
            print(f"Sending prompt to {self.model_name}: {prompt}")
        response: ChatResponse = chat(
            model=self.model_name,
            messages=[{'role': 'user', 'content': prompt}]
        )
        return response.message.content

    @property
    def _llm_type(self) -> str:
        return "ollama"

# Replace LlamaCpp with OllamaLLM
model = OllamaLLM(model_name="llama3.2", verbose=True)

In [ ]:
# query = "کاربرد سیستم های صف چیست؟"
context_list = tbl.search(query, query_type="hybrid").limit(5).to_list()
context_list

In [ ]:
context = ''.join([f"{c['text']}\n\n" for c in context_list])

print(context)

In [ ]:
system_prompt = "Answer user query based on the given context."
user_prompt = f"Question:\n{query}\nContext:\n{context}"
print(user_prompt)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(system_prompt),
    HumanMessage(user_prompt),
]

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts.chat import ChatPromptTemplate

template = """Answer user query based on the given context.

    Question:{Question}

    Context:
    {Context}"""

prompt = ChatPromptTemplate.from_template(template)
chain = prompt | model | StrOutputParser()

response2  = chain.invoke({
    "Question": query,
    "Context": context
})
response2

In [ ]:
response = gpt4o_chat.invoke(messages)
response.pretty_print()

In [ ]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='llama3.2', messages=[
  {
    'role': 'user',
    'content': query,
  },
])
print(response['message']['content'])

In [ ]:
# query = "کاربرد سیستم های صف چیست؟"
response = gpt4o_chat.invoke(query)
response.pretty_print()